In [ ]:
import numpy as np
import seaborn as sns
import requests
import pandas as pd
import seaborn as sns 
import matplotlib as plt 
import plotly.express as px

API Requesting and Creating the Dataframe using JSON, from 01/2010 - 06/2025

In [ ]:
# api fetching from eia data

#api_key = "your_api_key"
#url = f"https://api.eia.gov/v2/natural-gas/cons/sum/data/?api_key={api_key}&frequency=monthly&data[0]=value&start=2020-01&end=2025-06&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=15000"


#response = requests.get(url)
#data_2325 = response.json()

#print(data_2325.keys())   # top-level keys these are the keys being spitted out from below!

#records = data_2325["response"]["data"]

#df2425 = pd.DataFrame(records)
    

import requests
import pandas as pd
import time

api_key = "your_api_key"  # Replace with your EIA API key
base_url = "https://api.eia.gov/v2/natural-gas/cons/sum/data/?frequency=monthly&data[0]=value&start=2010-01&end=2025-06&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000"

all_data = []
offset = 0
length = 5000  # You can reduce this if the API limits responses

while True:
    # Prepare the request
    params = {
        "frequency": "monthly",
        "data[0]": "value",
        "start": "2010-01",
        "end": "2025-07",
        "sort[0][column]": "period",
        "sort[0][direction]": "asc",
        "offset": offset,
        "length": length,
        "api_key": api_key
    }

    response = requests.get(base_url, params=params)
    response.raise_for_status()  # Stops if there is an HTTP error
    data_json = response.json()

    # Extract the data
    data_batch = data_json.get("response", {}).get("data", [])
    if not data_batch:
        break  # No more data, exit loop

    all_data.extend(data_batch)
    offset += length

    print(f"Fetched {len(all_data)} rows so far...")

    # Optional: pause to avoid hitting API rate limits
    time.sleep(0.3)

# Convert all data into a DataFrame
dfnew = pd.DataFrame(all_data)
dfnew.tail(100)


In [ ]:
dfnew.info()



Converting Data Types!

In [ ]:
# default code to convert datatypes

dfnew['value'] = pd.to_numeric(df['value'], errors = 'coerce') # converting from object to numeric! 

df['period'] = pd.to_datetime(df['period'], errors = 'coerce') # converting from obeject to datetime!

# now will check the data types of the variables 
print(df.dtypes)

# also checking the missing values of the data set.
print(df.isnull().sum())

non_missing = df['value'].notna().sum()
print("Number of non-missing values:", non_missing)

# Mean consumption per state
mean_per_state = df.groupby('area-name')['value'].mean()
print(mean_per_state





In [ ]:


dfnew['value'] = pd.to_numeric(dfnew['value'], errors = 'coerce') # converting from object to numeric! 

dfnew['period'] = pd.to_datetime(dfnew['period'], errors = 'coerce') # converting from obeject to datetime!

# now will check the data types of the variables 
print(dfnew.dtypes)

# also checking the missing values of the data set.
print(dfnew.isnull().sum())

non_missing = dfnew['value'].notna().sum()
print("Number of non-missing values:", non_missing)

# Mean consumption per state
mean_per_state = dfnew.groupby('area-name')['value'].mean()
print(mean_per_state)

# views which rows are invald
#dfnew[dfnew.isnull().any(axis=1)]


Extracting Data to show month to month consumption from 01/2023 to 06/2025.

In [ ]:
# default code to filter by year and by month and y
df['year'] = df['period'].dt.year
# extracting month
df['month'] = df['period'].dt.month

# Filter to the date range (2023-01 to 2025-06)
#mask = (df['period'] >= "2023-01-01") & (df['period'] <= "2025-06-30")
#df_filtered = df.loc[mask].sort_values(['area-name', 'period'])

#print(df_filtered.head(20))


monthly_state = (
    df.groupby(['area-name', 'period', 'product', 'product-name', 
                'process', 'process-name', 'series', 'series-description', 'units'])['value']
      .sum()
      .reset_index()
      .sort_values(['area-name', 'period'])
)

print(monthly_state.info())

In [15]:
# actual applicaiton this is sorting the dataframe by the acutal month and year and not by state yet!!!!
## extract year
dfnew['year'] = dfnew['period'].dt.year
# extracting month
dfnew['month'] = dfnew['period'].dt.month

# Filter to the date range (2023-01 to 2025-06)
mask = (dfnew['period'] >= "2010-01-01") & (dfnew['period'] <= "2025-06-30")
df_filtered = dfnew.loc[mask].sort_values(['area-name', 'period'])

print(df_filtered.head())
df_filtered.info()
display(df_filtered.isnull().sum())


# seperating the null values to visualize whats happening!
missingrows = df_filtered[df_filtered['value'].isnull()]
non_missingrows = df_filtered[df_filtered['value'].notnull()]

display(f'total missing values', {missingrows.shape[0]})

# summarizing. by month
missing_bymonth = (
    missingrows.groupby(missingrows['period'].dt.to_period("M"))['value'] #df_nulls['period'].dt.to_period("M") Converts each datetime into a monthly period.
    .size()
    .reset_index(name='missing_count')
)

# period is currently an onject such as (period("2012-05"), ("M")) what we are doing is converting the period into a timestamp which makes it better for plotting
#
missing_bymonth['period'] = missing_bymonth['period'].dt.to_timestamp()

print(missing_bymonth['period'].dtype)
print(missing_bymonth['missing_count'].dtype)
print(missing_bymonth.tail(100))
print(missing_bymonth.info())

# Group by state and month, counting missing rows
pivot_missing = (
    missingrows.groupby(['area-name', missingrows['period'].dt.to_period("Y")])
    .size()  # counts all missing rows in each group
    .unstack(fill_value=0)  # fills months with 0 if no missing values
)

# Convert period to datetime for plotting
pivot_missing.columns = pivot_missing.columns.to_timestamp()

# Sort states alphabetically or by total missing values
pivot_missing = pivot_missing.sort_index()  # alphabetical by state

import plotly.express as px

fig = px.imshow(
    pivot_missing,
    labels=dict(x="Year", y="State", color="Missing Count"),
    x=pivot_missing.columns,
    y=pivot_missing.index,   # explicit y-axis labels
    color_continuous_scale="Reds",
    text_auto=True
)

fig.update_xaxes(side="top")  # optional: move x-axis to top
fig.update_yaxes(tickmode='linear')  # ensures all states are labeled
fig.update_layout(
    title="Missing Natural Gas Values by State and Year",
    autosize=False,
    width=1400,   # wider figure
    height=1200,  # taller figure
)
fig.show()


import plotly.express as px

# Select top states (e.g., DC + 4 others)
top_states = pivot_missing.sum(axis=1).sort_values(ascending=False).head(5).index
pivot_top = pivot_missing.loc[top_states]

# Convert to long format for Plotly line plot
pivot_long = pivot_top.reset_index().melt(id_vars='area-name', var_name='year', value_name='missing_count')

fig = px.line(
    pivot_long,
    x='year',
    y='missing_count',
    color='area-name',
    markers=True,
    title='Missing Natural Gas Values Trend for Top States (Yearly)',
    labels={'year':'Year', 'missing_count':'Missing Count', 'area-name':'State'}
)
fig.show()


# investigating further what type of nautral gas dc is missing values forn
missingdc = missingrows[missingrows['area-name'] == "USA-DC"]
non_missingrows = non_missingrows[non_missingrows['area-name'] == 'USA-DC']

# Filter non-missing rows to DC
observed_dc = non_missingrows[non_missingrows['area-name'] == "USA-DC"].copy()

# Convert period to month periods for animation
observed_dc['month_period'] = observed_dc['period'].dt.to_period('M').dt.to_timestamp()
# just for dc
animated_dc = (
    observed_dc.groupby(['month_period', 'process-name'])
    .size()
    .reset_index(name='count')  # observed counts
)

missinggastyp = missingdc.groupby(['product', 'product-name', 'process', 'process-name', 'series', 'series-description']).size().reset_index(name='missing_count')
non_missinggastyp = non_missingrows.groupby(['product', 'product-name', 'process', 'process-name', 'series', 'series-description']).size().reset_index(name='non_missing_count')

# sort by descending missing count
missinggastyp= missinggastyp.sort_values('missing_count', ascending=False)
missinggastyp.head(20)

#sorting by descending missing coutn
non_missinggastyp = non_missinggastyp.sort_values('non_missing_count', ascending=False)
non_missinggastyp.head(20)


# 1️⃣ Filter DC non-missing data
observed_dc = non_missingrows[non_missingrows['area-name'] == "USA-DC"].copy()
observed_dc['month_period'] = observed_dc['period'].dt.to_period('M').dt.to_timestamp()

# 2️⃣ Group by month and process
monthly_counts = (
    observed_dc.groupby(['month_period', 'process-name'])
    .size()
    .reset_index(name='count')
)

# 3️⃣ Create a complete timeline of months
all_months = pd.date_range(start='2010-01-01', end='2025-06-01', freq='MS')
all_processes = monthly_counts['process-name'].unique()
full_index = pd.MultiIndex.from_product([all_months, all_processes], names=['month_period', 'process-name'])

# 4️⃣ Reindex to include all months/processes, fill missing counts with 0
monthly_counts = monthly_counts.set_index(['month_period', 'process-name']).reindex(full_index, fill_value=0).reset_index()

# 5️⃣ Compute cumulative counts per process
monthly_counts['cumulative_count'] = monthly_counts.groupby('process-name')['count'].cumsum()

# 6️⃣ Create string version for animation frame
monthly_counts['month_period_str'] = monthly_counts['month_period'].dt.strftime('%Y-%m')


import plotly.express as px

fig = px.bar(
    monthly_counts,
    x='process-name',
    y='cumulative_count',
    color='process-name',
    animation_frame='month_period_str',
    range_y=[0, monthly_counts['cumulative_count'].max() + 10],
    title='Cumulative Natural Gas Counts in DC by Process Type (2010–2025)',
    labels={'cumulative_count':'Cumulative Count', 'process-name':'Process Type'}
)

fig.update_layout(width=1400, height=800)
fig.show()




        period duoarea   area-name product product-name process  \
87  2010-01-01     SCA  CALIFORNIA    EPG0  Natural Gas     VCS   
187 2010-01-01     SCA  CALIFORNIA    EPG0  Natural Gas     VEU   
231 2010-01-01     SCA  CALIFORNIA    EPG0  Natural Gas     VDV   
245 2010-01-01     SCA  CALIFORNIA    EPG0  Natural Gas     VRS   
254 2010-01-01     SCA  CALIFORNIA    EPG0  Natural Gas     VGT   

                   process-name        series  \
87       Commercial Consumption      N3020CA2   
187  Electric Power Consumption      N3045CA2   
231    Vehicle Fuel Consumption  NA1570_SCA_2   
245     Residential Consumption      N3010CA2   
254      Delivered to Consumers      N3060CA2   

                                    series-description     value units  year  \
87   Natural Gas Deliveries to Commercial Consumers...   27422.0  MMCF  2010   
187  California Natural Gas Deliveries to Electric ...   66583.0  MMCF  2010   
231  California Natural Gas Vehicle Fuel Consumptio...    1153

period                  0
duoarea                 0
area-name               0
product                 0
product-name            0
process                 0
process-name            0
series                  0
series-description      0
value                 584
units                   0
year                    0
month                   0
dtype: int64

'total missing values'

{584}

datetime64[ns]
int64
        period  missing_count
84  2017-02-01              3
85  2017-03-01              3
86  2017-04-01              3
87  2017-05-01              3
88  2017-06-01              3
..         ...            ...
179 2025-02-01             12
180 2025-03-01             18
181 2025-04-01             19
182 2025-05-01             21
183 2025-06-01             20

[100 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   period         184 non-null    datetime64[ns]
 1   missing_count  184 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 3.0 KB
None


In [ ]:
monthly_statenew = (
    dfnew.groupby(['area-name', 'period', 'product', 'product-name', 
                'process', 'process-name', 'series', 'series-description', 'units'])['value']
      .sum()
      .reset_index()
      .sort_values(['area-name', 'period'])
)

print(monthly_statenew.info())


print(monthly_statenew.isnull().sum())


In [ ]:
#new data frame with sorting by state

df_state_sorted = df.sort_values(['area-name', 'period'])
# display(df_state_sorted.head(25))
rows_per_year = df.groupby(df['period'].dt.year)['value'].count()
# print(rows_per_year)

df_sorted_new = df.sort_values(['period', 'area-name'])
display(df_sorted_new.tail(50))

unite = df_sorted_new['area-name'].unique()
unite.sort()
display(unite)







In [ ]:
# seperating the different states by region N,E,W,S

regions = {
    "North East": ['USA-CT','USA-ME','MASSACHUSETTS','USA-NH', 'USA-RI','USA-VT','NEW YORK','USA-PA',"USA-NJ"],
    "Midwest" :   ['USA-IL','USA-IN','USA-MI','OHIO','USA-WI','USA-IA','USA-KS','MINNESOTA','USA-MO','USA-NE','USA-ND','USA-SD'],
    "South" :     ['USA-DE','FLORIDA','USA-GA','USA-MD','USA-NC','USA-SC','USA-VA','USA-DC','USA-WV','USA-AL','USA-KY','USA-MS','USA-TN','USA-AR','USA-LA','USA-OK','TEXAS'],
    "West" :      ['USA-AZ','COLORADO','USA-ID','USA-MT','USA-NV','USA-NM','USA-UT','USA-WY','USA-AK','CALIFORNIA','USA-HI','USA-OR','WASHINGTON'],
    "Nationwide" : ['U.S.'] 
}

#defining as function that adds a new column "region" by state

#def get_region(state):   # whats happening here is that we give this function a single state e.g., "california" it loops through every (region,state) until it finds the region related to this state
#    for region, states in regions.items():
#        if state in states: # givesmme O(S) time complexity, how can we improve? 
#           return region
#    return "Unknown"

#df_sorted_new["region"] = df_sorted_new["area-name"].apply(get_region)

# sort of tweaking the algorithm!
# Convert values in regions dict into sets
#regions_opt = {region: set(states) for region, states in regions.items()}

#def get_region_fast(state):
#    for region, states in regions_opt.items(): # membership becomes 1
#        if state in states:
#            return region
#    return "Unknown"

# df_sorted_new["region_fast"] = df_sorted_new["area-name"].apply(get_region_fast)

# a mapping approach (vectorizing the df)
state_to_region = {state: region for region, states in regions.items() for state in states}
df_sorted_new["region_map"] = df_sorted_new["area-name"].map(state_to_region).fillna("Unknown")

# merging with the refernce dataframe!
# sql0style approach!
#region_df = pd.DataFrame(
#    [(state, region) for region, states in regions.items() for state in states],
#    columns=["area-name", "region"]
#)

# df_sorted_new = df_sorted_new.merge(region_df, on="area-name", how="left")
#df_sorted_new["region"].fillna("Unknown", inplace=True)

# Example: bar plot of rows per region
import matplotlib.pyplot as plt

df_sorted_new["region_map"].value_counts().plot(kind="bar", color="skyblue")
plt.ylabel("Number of Rows")
plt.title("Number of Rows by Region")
plt.show()

# If using the map + dictionary approach TO CHECK IF ALL STATES ARE IN THERE RESPECTIVE REGIONS
unknown_states = df_sorted_new[df_sorted_new['region_map'].isna() | 
                               (df_sorted_new['region_map'] == "Unknown")]

print(unknown_states[['area-name', 'region_map']])
df_sorted_new.info()

In [ ]:
import timeit

# Original apply function
# t1 = timeit.timeit(lambda: df_sorted_new["area-name"].apply(get_region), number=10)

# Optimized apply with sets
# t2 = timeit.timeit(lambda: df_sorted_new["area-name"].apply(get_region_fast), number=10)

# Map + dictionary
t3 = timeit.timeit(lambda: df_sorted_new["area-name"].map(state_to_region).fillna("Unknown"), number=10)

#print(f"Apply + list lookup: {t1:.4f} sec")
#print(f"Apply + set lookup: {t2:.4f} sec")
print(f"Map + dictionary:   {t3:.4f} sec")


import matplotlib.pyplot as plt

#methods = ["Apply+List", "Apply+Set", "Map+Dict"]
#times = [t1, t2, t3]  # from your timeit runs

#plt.bar(methods, times)
#plt.ylabel("Execution Time (sec, lower is better)")
#plt.title("Categorization Performance by Method")
#plt.show()


In [ ]:
import pandas as pd

# Example sizes
dataset_sizes = [1000, 5000, 10000, 50000,100000]  # number of rows

# List to store smaller or larger DataFrames
dfs_scaled = []

for n in dataset_sizes:
    if n <= len(df_sorted_new):
        # Sample n rows if smaller than base
        dfs_scaled.append(df_sorted_new.sample(n, replace=False).reset_index(drop=True))
    else:
        # Repeat rows if n > len(base df) to simulate larger datasets
        repeats = n // len(df_sorted_new) + 1
        df_large = pd.concat([df_sorted_new] * repeats, ignore_index=True)
        dfs_scaled.append(df_large.head(n))

import timeit

# Lists to store execution times
apply_list_times = []
apply_set_times = []
map_dict_times = []

# Prepare optimized set and dict
regions_set = {region: set(states) for region, states in regions.items()}
state_to_region = {state: region for region, states in regions.items() for state in states}

#for df_temp in dfs_scaled:
    # Apply + List
    #t1 = timeit.timeit(lambda: df_temp["area-name"].apply(get_region), number=1)
   # apply_list_times.append(t1)
    
    # Apply + Set
    #t2 = timeit.timeit(lambda: df_temp["area-name"].apply(lambda x: next((r for r,s in regions_set.items() if x in s), "Unknown")), number=1)
    #apply_set_times.append(t2)
    
    # Map + Dict
    #t3 = timeit.timeit(lambda: df_temp["area-name"].map(state_to_region).fillna("Unknown"), number=1)
    #map_dict_times.append(t3)


import matplotlib.pyplot as plt

#plt.plot(dataset_sizes, apply_list_times, marker='o', label='Apply + List')
#plt.plot(dataset_sizes, apply_set_times, marker='s', label='Apply + Set')
#plt.plot(dataset_sizes, map_dict_times, marker='^', label='Map + Dict')
#plt.xlabel('Dataset Size (number of rows)')
#plt.ylabel('Execution Time (seconds)')
#plt.title('Categorization Algorithm Performance Scaling')
#plt.legend()
#plt.grid(True)
#plt.show()


Creating the Interactive Display!

In [ ]:
# importing the necessary libraries to build the dashboard
from dash import Dash, html, dash_table, dcc
import plotly.express as px

# exmaple app layout or at least one piece!
# app.layout = [html.Div(children='Hello World')]
# children is often used as represents the app components, probably used for text content beind displayed onto the webpage.

df = df_sorted_new.copy()

# initilizing the app
app = Dash(__name__)  # dash constructor! (responsible for initializing the app)

# basic app layout
app.layout = [
    html.Div(children="My First Interactive MAPP"),
    dash_table.DataTable(data=df.to_dict('records'), page_size=10),
    dcc.Graph(figure=px.histogram(df, x = 'region_map', y = 'value', histfunc='avg'))
]

# running the app
if __name__ == '__main__':
    app.run(debug=True)

 

In [ ]:
from dash import Dash, html, dash_table, dcc, Input, Output
import plotly.express as px
import pandas as pd

# Example data (replace with your df_sorted_new)
# Must contain: ['month', 'state', 'region_map', 'gas_type', 'value']
df = df_sorted_new.copy()
# Make sure you have this before the layout
df['month'] = df['period'].dt.to_period('M')  # YYYY-MM period
unique_months = df['month'].drop_duplicates().sort_values()


# Initialize the app
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H2("US Natural Gas Consumption Dashboard"),
    

  
# Then in your layout:
html.Div([
    html.Label("Select Month:"),
    dcc.Dropdown(
        id="month-filter",
        options=[
            {"label": m.strftime('%b %Y'), "value": str(m)} 
            for m in unique_months
        ],
        value=str(unique_months.iloc[0]),  # default first month
        clearable=False
    ),
], style={"width": "30%", "display": "inline-block", "padding": "10px"}),


    html.Div([
        html.Label("Select State:"),
        dcc.Dropdown(
            id="area-filter",
            options=[{"label": s, "value": s} for s in sorted(df["area-name"].unique())],
            value=None,  # default = all states
            multi=True,  # allow multiple states
            placeholder="Choose state(s)"
        ),
    ], style={"width": "30%", "display": "inline-block", "padding": "10px"}),

    html.Div([
        html.Label("Select Gas Type:"),
        dcc.RadioItems(
            id="process-filter",
            options=[{"label": g, "value": g} for g in sorted(df["process"].unique())],
            value=sorted(df["process"].unique())[0],
            inline=True
        ),
    ], style={"padding": "10px"}),

    html.Hr(),

    # Table + Graph
    dash_table.DataTable(
        id="data-table",
        page_size=10,
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left"}
    ),

    dcc.Graph(id="region-trend"),

    dcc.Graph(id="nationwide-trend")
])
from pandas import Period

# Callbacks
@app.callback(
    [Output("data-table", "data"),
     Output("region-trend", "figure"),
     Output("nationwide-trend", "figure")],
    [Input("month-filter", "value"),
     Input("area-filter", "value"),
     Input("process-filter", "value")]
)
def update_dashboard(month_str, area_name, process):
    #converting month string from dropdown back to period.
    month_period = Period(month_str, freq = 'M')

    
    
    # Filter by month + gas type
    dff = df[(df["month"] == month_period) & (df["process"] == process)]
  # Filter by state if selected
    if area_name:
        dff = dff[dff["area-name"].isin(area_name)]
 

    # Table data
    table_data = dff.to_dict("records")
    
    
    month_label = month_period.strftime("%b %Y")

    # Regional consumption bar chart
    fig_region = px.bar(
        dff, x="region_map", y="value",
        color="region_map",
        title=f"Regional Avg Consumption - {month_label} ({process})",
        labels={"value": "Consumption"}
    )

 # Nationwide line chart
    df_nation = df[df["process"] == process].groupby("month")["value"].sum().reset_index()
    df_nation['month_label'] = df_nation['month'].dt.strftime("%b %Y")  # convert Periods to strings
    fig_nation = px.line(
        df_nation, x="month_label", y="value",
        title=f"Nationwide Monthly Consumption Trend ({process})",
        markers=True
    )

    return table_data, fig_region, fig_nation


if __name__ == "__main__":
    app.run(debug=True)
